## Initialize the spark environment and load the helper functions we have provided

In [1]:
import sys
sys.path.append('/home/student/ROI/SparkProgram')

import pandas as pd
import matplotlib as mp
import numpy
from matplotlib import pyplot as plt
#from IPython.display import display

import pyspark_helpers as pyh
sc, spark, conf = pyh.initspark()
from pyspark_helpers import display

initializing pyspark
pyspark initialized


## Read in a simple dataset of latitudes and longitudes

In [68]:
#filename = 'avocado.csv'
#filename = 'HousingData.csv'
filename = 'boston.csv'
df = spark.read.csv(f'/home/student/ROI/Spark/{filename}', header = True, inferSchema = True)
display(df)
df.printSchema()

# Save a pointer to the raw data
dfRaw = df



,TOWN,TRACT,LON,LAT,MEDV,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO
0,Nahant,2011,-70.9550,42.2550,24.0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3
1,Swampscott,2021,-70.9500,42.2875,21.6,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8
2,Swampscott,2022,-70.9360,42.2830,34.7,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8
3,Marblehead,2031,-70.9280,42.2930,33.4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7
4,Marblehead,2032,-70.9220,42.2980,36.2,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7
5,Marblehead,2033,-70.9165,42.3040,28.7,0.02985,0.0,2.18,0,0.458,6.430,58.7,6.0622,3,222,18.7
6,Salem,2041,-70.9360,42.2970,22.9,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2
7,Salem,2042,-70.9375,42.3100,22.1,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311,15.2
8,Salem,2043,-70.9330,42.3120,16.5,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311,15.2
9,Salem,2044,-70.9290,42.3160,18.9,0.17004,12.5,7.87,0,0.524,6.004,85.9,6.5921,5,311,15.2


root
 |-- TOWN: string (nullable = true)
 |-- TRACT: integer (nullable = true)
 |-- LON: double (nullable = true)
 |-- LAT: double (nullable = true)
 |-- MEDV: double (nullable = true)
 |-- CRIM: double (nullable = true)
 |-- ZN: double (nullable = true)
 |-- INDUS: double (nullable = true)
 |-- CHAS: integer (nullable = true)
 |-- NOX: double (nullable = true)
 |-- RM: double (nullable = true)
 |-- AGE: double (nullable = true)
 |-- DIS: double (nullable = true)
 |-- RAD: integer (nullable = true)
 |-- TAX: integer (nullable = true)
 |-- PTRATIO: double (nullable = true)



In [69]:
print(df.columns)

['TOWN', 'TRACT', 'LON', 'LAT', 'MEDV', 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO']


In [81]:
if filename == 'avocado.csv':
    df = dfRaw.withColumnRenamed('4046','PLU4046').withColumnRenamed('4225','PLU4225').withColumnRenamed('4770','PLU4770')
    df.createOrReplaceTempView('dfRaw')
    df.printSchema()

    sql = '''select AveragePrice as target, `Total Volume` as totalvolume
    , PLU4046, PLU4225, PLU4770
    , `Small Bags` as smallbags, `Large Bags` as largebags, `XLarge Bags` as xlargebags
    , type, year, region
    FROM dfRaw'''

    df = spark.sql(sql)
    print(df)

    numeric_features = ['totalvolume','PLU4046', 'PLU4225', 'PLU4770', 'smallbags', 'largebags', 'xlargebags']
    categorical_features = ['type', 'year','region']
    target_label = 'target'
    print(df.take(1))
else:
    numeric_features = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO']
    categorical_features = [] #['TOWN', 'TRACT']
    target_label = 'MEDV'
    df = dfRaw.select(categorical_features + numeric_features + [target_label])
    df.printSchema()

print ('******')
display(df.describe())
print ('******')
display(df)

root
 |-- CRIM: double (nullable = true)
 |-- ZN: double (nullable = true)
 |-- INDUS: double (nullable = true)
 |-- CHAS: integer (nullable = true)
 |-- NOX: double (nullable = true)
 |-- RM: double (nullable = true)
 |-- AGE: double (nullable = true)
 |-- DIS: double (nullable = true)
 |-- RAD: integer (nullable = true)
 |-- TAX: integer (nullable = true)
 |-- PTRATIO: double (nullable = true)
 |-- MEDV: double (nullable = true)

******


,summary,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,MEDV
0,count,506,506,506,506,506,506,506,506,506,506,506,506
1,mean,3.6135235573122535,11.363636363636363,11.136778656126504,0.0691699604743083,0.5546950592885372,6.284634387351787,68.57490118577078,3.795042687747034,9.549407114624506,408.2371541501976,18.455533596837967,22.528853754940737
2,stddev,8.601545105332491,23.32245299451514,6.860352940897589,0.2539940413404101,0.11587767566755584,0.7026171434153232,28.148861406903595,2.10571012662761,8.707259384239366,168.53711605495903,2.1649455237144455,9.182175882166778
3,min,0.00632,0.0,0.46,0,0.385,3.561,2.9,1.1296,1,187,12.6,5.0
4,max,88.9762,100.0,27.74,1,0.871,8.78,100.0,12.1265,24,711,22.0,50.0


******


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,36.2
5,0.02985,0.0,2.18,0,0.458,6.430,58.7,6.0622,3,222,18.7,28.7
6,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2,22.9
7,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311,15.2,22.1
8,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311,15.2,16.5
9,0.17004,12.5,7.87,0,0.524,6.004,85.9,6.5921,5,311,15.2,18.9


## Turn the dataframe into vectors



In [82]:
import imp
imp.reload(pyh)

# df10 = pyh.StringIndexEncode(df, categorical_features)
# display(df10)
# df11 = pyh.OneHotEncode(df10, categorical_features)
# display(df11)
# df12 = pyh.AssembleFeatures(df11, categorical_features, numeric_features, 'target', False)
# display(df12)

dfML = pyh.MakeMLDataFrame(df, categorical_features, numeric_features, target_label, False)
display(dfML)
dfML.printSchema()


,target,features
0,24.0,"[0.00632, 18.0, 2.31, 0.0, 0.538, 6.575, 65.2,..."
1,21.6,"[0.02731, 0.0, 7.07, 0.0, 0.469, 6.421, 78.9, ..."
2,34.7,"[0.02729, 0.0, 7.07, 0.0, 0.469, 7.185, 61.1, ..."
3,33.4,"[0.03237, 0.0, 2.18, 0.0, 0.458, 6.998, 45.8, ..."
4,36.2,"[0.06905, 0.0, 2.18, 0.0, 0.458, 7.147, 54.2, ..."
5,28.7,"[0.02985, 0.0, 2.18, 0.0, 0.458, 6.43, 58.7, 6..."
6,22.9,"[0.08829, 12.5, 7.87, 0.0, 0.524, 6.012, 66.6,..."
7,22.1,"[0.14455, 12.5, 7.87, 0.0, 0.524, 6.172, 96.1,..."
8,16.5,"[0.21124, 12.5, 7.87, 0.0, 0.524, 5.631, 100.0..."
9,18.9,"[0.17004, 12.5, 7.87, 0.0, 0.524, 6.004, 85.9,..."


root
 |-- target: double (nullable = true)
 |-- features: vector (nullable = true)



## Split the dataset into train and test

In [83]:
train, test = dfML.randomSplit([.7,.3], seed = 1000)
print (f'Training set row count {train.count()}')
print (f'Testing set row count {test.count()}')



Training set row count 366
Testing set row count 140


## Run Linear Regression

In [84]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='target', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(train)
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

trainingSummary = lrModel.summary
print("Root Mean Squared Error: {}\nR Squared (R2) {}".format(trainingSummary.rootMeanSquaredError, trainingSummary.r2))


Coefficients: [-0.11629553483841598,0.0194623555816696,-0.05341530387639158,1.9016965092032956,-11.334191943299343,6.531078457188811,-0.032237321194478516,-0.7503624062693484,0.0,-0.001515529789740807,-0.7359846768990195]
Intercept: 7.849767103202408
Root Mean Squared Error: 5.398671038650852
R Squared (R2) 0.6366985811998482


## Run test data


In [107]:
lrPredictions = lrModel.transform(test)
display(lrPredictions.select("prediction","target","features"), 30)
from pyspark.ml.evaluation import RegressionEvaluator
lrEvaluator = RegressionEvaluator(predictionCol="prediction", labelCol="target",metricName="r2")
testResult = lrModel.evaluate(test)
print("Root Mean Squared Error: {}".format(testResult.rootMeanSquaredError))

,prediction,target,features
0,14.925031,5.6,"[25.0461, 0.0, 18.1, 0.0, 0.693, 5.987, 100.0,..."
1,2.878015,7.0,"[45.7461, 0.0, 18.1, 0.0, 0.693, 4.519, 100.0,..."
2,18.628179,7.2,"[18.0846, 0.0, 18.1, 0.0, 0.679, 6.434, 100.0,..."
3,9.075642,7.4,"[22.5971, 0.0, 18.1, 0.0, 0.7, 5.0, 89.5, 1.51..."
4,22.052172,7.5,"[10.8342, 0.0, 18.1, 0.0, 0.679, 6.782, 90.8, ..."
5,16.373493,8.2,"[15.1772, 0.0, 18.1, 0.0, 0.74, 6.152, 100.0, ..."
6,22.250823,8.4,"[11.8123, 0.0, 18.1, 0.0, 0.718, 6.824, 76.5, ..."
7,15.590893,8.4,"[13.6781, 0.0, 18.1, 0.0, 0.74, 5.935, 87.9, 1..."
8,10.486656,8.5,"[41.5292, 0.0, 18.1, 0.0, 0.693, 5.531, 85.4, ..."
9,9.088412,8.8,"[73.5341, 0.0, 18.1, 0.0, 0.679, 5.957, 100.0,..."


Root Mean Squared Error: 5.599620898642493


## Try Decision Tree Regression

In [101]:
from pyspark.ml.regression import DecisionTreeRegressor
dt = DecisionTreeRegressor(featuresCol ='features', labelCol = 'target')
dtModel = dt.fit(train)
dtPredictions = dtModel.transform(test)
important = dtModel.featureImportances
print(type(important), important)
#importantDict = zip(important[1], important[2])
#print (importantDict)
print (important[3])
from pyspark.ml.evaluation import RegressionEvaluator
dtEvaluator = RegressionEvaluator(predictionCol="prediction", labelCol="target",metricName="rmse")
testResult = dtEvaluator.evaluate(dtPredictions)
print("Root Mean Squared Error: {}".format(testResult))
dfML.take(1)

<class 'pyspark.ml.linalg.SparseVector'> (11,[0,4,5,6,7,9,10],[0.09979170814256355,0.141872377415381,0.6646056871858018,0.035443837056989606,0.050146175026264186,0.003244702319768674,0.004895512853231193])
0.0
Root Mean Squared Error: 6.47579033173964


[Row(target=24.0, features=DenseVector([0.0063, 18.0, 2.31, 0.0, 0.538, 6.575, 65.2, 4.09, 1.0, 296.0, 15.3]))]

## Try Gradient Booste Tree

In [106]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol = 'features', labelCol = 'target', maxIter=30)
gbtModel = gbt.fit(train)
gbtPredictions = gbtModel.transform(test)
display(gbtPredictions.select('prediction', 'target', 'features'), 20)
from pyspark.ml.evaluation import RegressionEvaluator
gbtEvaluator = RegressionEvaluator(predictionCol="prediction", labelCol="target",metricName="rmse")
testResult = gbtEvaluator.evaluate(gbtPredictions)
print("Root Mean Squared Error: {}".format(testResult))



,prediction,target,features
0,9.275024,5.6,"[25.0461, 0.0, 18.1, 0.0, 0.693, 5.987, 100.0,..."
1,8.418278,7.0,"[45.7461, 0.0, 18.1, 0.0, 0.693, 4.519, 100.0,..."
2,9.132312,7.2,"[18.0846, 0.0, 18.1, 0.0, 0.679, 6.434, 100.0,..."
3,11.945834,7.4,"[22.5971, 0.0, 18.1, 0.0, 0.7, 5.0, 89.5, 1.51..."
4,12.973586,7.5,"[10.8342, 0.0, 18.1, 0.0, 0.679, 6.782, 90.8, ..."
5,10.317538,8.2,"[15.1772, 0.0, 18.1, 0.0, 0.74, 6.152, 100.0, ..."
6,13.232848,8.4,"[11.8123, 0.0, 18.1, 0.0, 0.718, 6.824, 76.5, ..."
7,13.747541,8.4,"[13.6781, 0.0, 18.1, 0.0, 0.74, 5.935, 87.9, 1..."
8,9.813878,8.5,"[41.5292, 0.0, 18.1, 0.0, 0.693, 5.531, 85.4, ..."
9,8.986420,8.8,"[73.5341, 0.0, 18.1, 0.0, 0.679, 5.957, 100.0,..."


Root Mean Squared Error: 6.143094237169346
